In [1801]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import glob
import re
import datetime
from datetime import datetime

import scipy as sc
from pandas import read_csv

In [1802]:
df_other_train = read_csv("data/other_train.csv")
df_personal_train = read_csv("data/personal_train.csv")
df_other_valid = read_csv("data/other_valid.csv")
df_personal_valid = read_csv("data/personal_valid.csv")

df_merged1 = pd.merge(df_personal_train, df_other_train, on=['name', 'address'])
df_merged2 = pd.merge(df_other_valid, df_personal_valid, on=['name', 'address'])

df_merged1 = df_merged1.rename(columns={'Unnamed: 0_x': 'index',})
df_merged1

,index,name,address,age,sex,date_of_birth,Unnamed: 0_y,race,marital-status,occupation,...,capital-gain,education,fnlwgt,class,income,medical_info,native-country,hours-per-week,capital-loss,workclass
0,0,Roscoe Bohannon,"7183 Osborne Ways Apt. 651\r\nEast Andrew, OH ...",58,Male,1961-04-27,717,White,Married-civ-spouse,Transport-moving,...,0.0,Assoc-voc,241259.0,0.0,<=50K,"{'mean_glucose':'132.9296875','std_glucose':'6...",United-States,40.0,0.0,Private
1,1,Ernest Kline,"391 Ball Road Suite 961\r\nFlowersborough, IN ...",43,Female,1976-09-12,3106,Black,Divorced,Other-service,...,0.0,HS-grad,172232.0,0.0,<=50K,"{'mean_glucose':'103.1171875','std_glucose':'-...",United-States,30.0,0.0,Private
2,2,Harold Hendriks,"8702 Vincent Square\r\nNew Jerryfurt, CO 30614",51,Male,1967-11-12,3601,White,Married-civ-spouse,Exec-managerial,...,0.0,Some-college,345705.0,0.0,<=50K,"{'mean_glucose':'125.5390625','std_glucose':'5...",United-States,40.0,0.0,Private
3,3,Randy Baptiste,"2751 Harris Crossroad\r\nWest Ashley, CA 30311",66,Male,1953-06-02,3209,Amer-Indian-Eskimo,Never-married,Handlers-cleaners,...,0.0,HS-grad,149297.0,0.0,<=50K,"{'mean_glucose':'136.0859375','std_glucose':'4...",United-States,30.0,0.0,Private
4,4,Anthony Colucci,"904 Robert Cliffs Suite 186\r\nWest Kyle, CO 7...",45,Female,1974/09/11,1632,White,Never-married,Craft-repair,...,0.0,Some-college,253262.0,0.0,<=50K,"{'mean_glucose':'99.328125','std_glucose':'38....",United-States,40.0,0.0,Private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,3928,Clair Pinkleton,"8582 Lee Roads\r\nCostaburgh, SC 21897",53,Female,1966-04-29,2554,White,Never-married,Adm-clerical,...,0.0,Bachelors,368739.0,0.0,<=50K,"{'mean_glucose':'161.4453125','std_glucose':'n...",United-States,40.0,0.0,Private
3979,3929,William Pope,68430 Erik Terrace Suite 279\r\nPort Adamborou...,81,Male,1938-10-24 00:00:00,2375,White,Never-married,Other-service,...,0.0,HS-grad,387430.0,1.0,<=50K,"{'mean_glucose':'114.1484375','std_glucose':'5...",United-States,18.0,0.0,private
3980,3930,Jared Kinsey,Unit 8416 Box 3801\r\nDPO AE 31166,46,Female,1973-02-23,2871,Black,Divorced,Adm-clerical,...,0.0,Bachelors,137304.0,0.0,<=50K,"{'mean_glucose':'128.734375','std_glucose':'57...",United-States,40.0,0.0,Private
3981,3931,Frederick Lawley,"34420 Brian Stream\r\nLake Michaeltown, MS 01832",46,Female,1973-08-09 00:00:00,3127,White,Never-married,Other-service,...,0.0,10th,65225.0,1.0,<=50K,"{'mean_glucose':'70.7421875','std_glucose':'44...",United-States,40.0,0.0,Private


In [1803]:
# df_merged2 = df_merged2.rename(columns={'Unnamed: 0_x': 'index',})
# df_merged2['index'] = df_merged2['index'] + df_merged1.shape[0]
# df_merged2['index']

In [1804]:
df_merged1 = df_merged1.drop('index', 1)
df_merged1 = df_merged1.drop('Unnamed: 0_y', 1)

In [1805]:
df_merged1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 3982
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            3983 non-null   object 
 1   address         3983 non-null   object 
 2   age             3983 non-null   int64  
 3   sex             3983 non-null   object 
 4   date_of_birth   3983 non-null   object 
 5   race            3691 non-null   object 
 6   marital-status  3970 non-null   object 
 7   occupation      3961 non-null   object 
 8   pregnant        3969 non-null   object 
 9   education-num   3961 non-null   float64
 10  relationship    3954 non-null   object 
 11  capital-gain    3969 non-null   float64
 12  education       3972 non-null   object 
 13  fnlwgt          3968 non-null   float64
 14  class           3960 non-null   float64
 15  income          3972 non-null   object 
 16  medical_info    3966 non-null   object 
 17  native-country  3966 non-null   o

In [1806]:
# def replace_value(data_frame):
#     aggregation_functions = {'name': 'first', 'address': 'first', 'race': 'first', 'marital-status': 'first', 'occupation': 'first',
#                              'pregnant': 'first', 'education-num': 'first', 'relationship': 'first', 'capital-gain': 'first',
#                              'education': 'first', 'fnlwgt': 'first', 'class': 'first', 'income': 'first', 'medical_info': 'first',
#                             'native-country': 'first', 'hours-per-week': 'first', 'capital-loss': 'first', 'workclass': 'first'}
#     return data_frame.groupby(data_frame['name'], as_index=False).aggregate(aggregation_functions)

def replace_value(data_frame):
    aggregation_functions = {}
    for col in data_frame.columns:
        aggregation_functions[col] = "first" 

    return data_frame.groupby(data_frame['name'], as_index=False).aggregate(aggregation_functions)


In [1807]:
df_merged1 = replace_value(df_merged1)

In [1808]:
df_merged1.shape[0]

3933

In [1809]:
df_merged = df_merged1.merge(df_merged2, how='outer')

df_merged = df_merged.rename(columns={'Unnamed: 0_x': 'index',})
df_merged = df_merged.drop('Unnamed: 0_y', 1)
df_merged = df_merged.drop('index', 1)
df_merged

,name,address,age,sex,date_of_birth,race,marital-status,occupation,pregnant,education-num,...,capital-gain,education,fnlwgt,class,income,medical_info,native-country,hours-per-week,capital-loss,workclass
0,Aaron Davis,"379 Rodriguez Inlet\r\nFisherbury, MD 83114",37,Female,1982/02/08,White,Divorced,Other-service,f,9.0,...,0.0,HS-grad,203463.0,0.0,<=50K,"{'mean_glucose':'130.640625','std_glucose':'na...",United-States,40.0,0.0,Private
1,Aaron Dini,"105 Medina Springs Suite 462\r\nEmilyland, KS ...",58,Female,1961-05-04,White,Never-married,Other-service,f,9.0,...,0.0,HS-grad,166744.0,0.0,<=50K,"{'mean_glucose':'99.828125','std_glucose':'47....",United-States,38.0,0.0,Private
2,Aaron Gann,59685 Thomas Extension Apt. 142\r\nFieldsborou...,48,Male,1971-02-03 00 00 00,White,Married-civ-spouse,Handlers-cleaners,f,10.0,...,0.0,Some-college,165517.0,0.0,<=50K,"{'mean_glucose':'92.7734375','std_glucose':'34...",United-States,40.0,0.0,Private
3,Aaron Gregg,"10883 Williams Locks Suite 821\r\nEast Carmen,...",45,Male,1973-12-23,White,Married-civ-spouse,Farming-fishing,f,4.0,...,0.0,7th-8th,22907.0,0.0,<=50K,"{'mean_glucose':'101.6953125','std_glucose':'4...",United-States,40.0,0.0,Self-emp-not-inc
4,Aaron Lindley,"564 Stevenson Park Apt. 839\r\nRodriguezville,...",56,Male,1963-08-22,White,Divorced,Transport-moving,f,10.0,...,0.0,Some-college,88019.0,0.0,<=50K,"{'mean_glucose':'114.8359375','std_glucose':'5...",United-States,55.0,0.0,Private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,Arturo Mckee,"64582 Mcgee Orchard\r\nBridgeschester, VT 81156",60,Male,1959-09-21,White,Married-civ-spouse,Prof-specialty,f,15.0,...,0.0,Prof-school,132601.0,1.0,>50K,"{'mean_glucose':'28.1953125','std_glucose':'39...",United-States,40.0,0.0,Private
5290,Bennie Jones,"9406 Desiree Rest\r\nSouth Laurafort, ME 72779",43,Male,76-09-04,White,Married-civ-spouse,Transport-moving,FALSE,9.0,...,0.0,HS-grad,85434.0,0.0,<=50K,"{'mean_glucose':'110.0546875','std_glucose':'4...",United-States,40.0,0.0,Private
5291,Bobby Hawk,"41416 Ellis Island\r\nEdgarchester, OR 73214",50,Male,1968-12-02,White,Married-civ-spouse,Prof-specialty,f,13.0,...,5178.0,Bachelors,174575.0,0.0,>50K,"{'mean_glucose':'159.078125','std_glucose':'39...",United-States,40.0,0.0,Local-gov
5292,Wally Suggs,"624 Jeremiah Inlet Suite 082\r\nEast Jerome, H...",49,Female,1970-04-05,White,Divorced,Sales,f,9.0,...,0.0,HS-grad,148903.0,0.0,<=50K,"{'mean_glucose':'116.46875','std_glucose':'35....",United-States,40.0,0.0,private


In [1810]:
# dir_in = 'data'
# filenames = [i for i in glob.glob(dir_in + '/*')]
# print(*filenames, sep='\n')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5294 entries, 0 to 5293
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            5294 non-null   object 
 1   address         5294 non-null   object 
 2   age             5294 non-null   int64  
 3   sex             5294 non-null   object 
 4   date_of_birth   5294 non-null   object 
 5   race            4912 non-null   object 
 6   marital-status  5280 non-null   object 
 7   occupation      5276 non-null   object 
 8   pregnant        5275 non-null   object 
 9   education-num   5275 non-null   float64
 10  relationship    5271 non-null   object 
 11  capital-gain    5274 non-null   float64
 12  education       5278 non-null   object 
 13  fnlwgt          5282 non-null   float64
 14  class           5278 non-null   float64
 15  income          5273 non-null   object 
 16  medical_info    5279 non-null   object 
 17  native-country  5281 non-null   o

In [1811]:
medical_line = df_merged["medical_info"].str.replace(':', ',').str.replace("'", '').str.replace("{", '').str.replace("}", '')
medical_elements = medical_line.str.split(",", -1, expand = True)

diabetes_dataset = {}

for index in range(0, medical_elements.shape[1]):
    i = medical_elements[0][0]
    if(index % 2 == 0):
        diabetes_dataset[medical_elements[index][0]] = medical_elements[index + 1]
        
diabetes_dataset = pd.DataFrame(diabetes_dataset)
df_merged = pd.concat([df_merged, diabetes_dataset.reindex(df_merged.index)], axis = 1)
df_merged = df_merged.drop(columns = ['medical_info'])

In [1812]:
df_merged['mean_glucose'] = df_merged['mean_glucose'].astype(float)
df_merged['std_glucose'] = df_merged['std_glucose'].astype(float)
df_merged['kurtosis_glucose'] = df_merged['kurtosis_glucose'].astype(float)
df_merged['skewness_glucose'] = df_merged['skewness_glucose'].astype(float)
df_merged['mean_oxygen'] = df_merged['mean_oxygen'].astype(float)
df_merged['std_oxygen'] = df_merged['std_oxygen'].astype(float)
df_merged['kurtosis_oxygen'] = df_merged['kurtosis_oxygen'].astype(float)
df_merged['skewness_oxygen'] = df_merged['skewness_oxygen'].astype(float)

In [1813]:
# df_merged.set_index('index', inplace=True)
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5294 entries, 0 to 5293
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              5294 non-null   object 
 1   address           5294 non-null   object 
 2   age               5294 non-null   int64  
 3   sex               5294 non-null   object 
 4   date_of_birth     5294 non-null   object 
 5   race              4912 non-null   object 
 6   marital-status    5280 non-null   object 
 7   occupation        5276 non-null   object 
 8   pregnant          5275 non-null   object 
 9   education-num     5275 non-null   float64
 10  relationship      5271 non-null   object 
 11  capital-gain      5274 non-null   float64
 12  education         5278 non-null   object 
 13  fnlwgt            5282 non-null   float64
 14  class             5278 non-null   float64
 15  income            5273 non-null   object 
 16  native-country    5281 non-null   object 


In [1814]:
df_merged['pregnant'].unique()

array(['f', 'FALSE', 'F', 't', 'TRUE', 'T', nan], dtype=object)

In [1815]:
def cleanse_some_values(data_frame, column, change_from, change_to):
    for i in range(len(change_from)):
        data_frame.loc[data_frame[column] == change_from[i], column] = change_to

In [1816]:
cleanse_some_values(df_merged, 'pregnant', ["F", "FALSE"], "f")
cleanse_some_values(df_merged, 'pregnant', ["T", "TRUE"], "t")
# cleanse_some_values(df_merged, 'pregnant', "FALSE", "f")
# cleanse_some_values(df_merged, 'pregnant', "T", "t")
# cleanse_some_values(df_merged, 'pregnant', "TRUE", "t")

In [1817]:
df_merged['marital-status'].unique()

array([' Divorced', ' Never-married', ' Married-civ-spouse', ' Widowed',
       ' Separated', ' Married-spouse-absent', ' Married-AF-spouse', nan],
      dtype=object)

In [1818]:
cleanse_some_values(df_merged, 'marital-status', [" Widowed", " Never-married", " Divorced"], "single")
cleanse_some_values(df_merged, 'marital-status', [" Married-civ-spouse", " Separated", " Married-AF-spouse", " Married-spouse-absent"], "married")

In [1819]:
df_merged

,name,address,age,sex,date_of_birth,race,marital-status,occupation,pregnant,education-num,...,capital-loss,workclass,mean_glucose,std_glucose,kurtosis_glucose,skewness_glucose,mean_oxygen,std_oxygen,kurtosis_oxygen,skewness_oxygen
0,Aaron Davis,"379 Rodriguez Inlet\r\nFisherbury, MD 83114",37,Female,1982/02/08,White,single,Other-service,f,9.0,...,0.0,Private,130.640625,NaN,-0.278638,-0.200794,11.642977,41.904978,3.602677,11.954904
1,Aaron Dini,"105 Medina Springs Suite 462\r\nEmilyland, KS ...",58,Female,1961-05-04,White,single,Other-service,f,9.0,...,0.0,Private,99.828125,47.239766,0.469564,0.767540,2.816054,16.659173,7.999821,78.263172
2,Aaron Gann,59685 Thomas Extension Apt. 142\r\nFieldsborou...,48,Male,1971-02-03 00 00 00,White,married,Handlers-cleaners,f,10.0,...,0.0,Private,92.773438,34.005793,0.602241,3.485345,1.086957,12.610281,14.133424,224.144386
3,Aaron Gregg,"10883 Williams Locks Suite 821\r\nEast Carmen,...",45,Male,1973-12-23,White,married,Farming-fishing,f,4.0,...,0.0,Self-emp-not-inc,101.695312,47.525414,0.402180,0.239359,2.717391,19.675471,8.112776,71.312964
4,Aaron Lindley,"564 Stevenson Park Apt. 839\r\nRodriguezville,...",56,Male,1963-08-22,White,single,Transport-moving,f,10.0,...,0.0,Private,114.835938,51.926146,-0.009627,-0.528806,1.227425,12.031249,14.087309,234.473686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,Arturo Mckee,"64582 Mcgee Orchard\r\nBridgeschester, VT 81156",60,Male,1959-09-21,White,married,Prof-specialty,f,15.0,...,0.0,Private,28.195312,3921.376883,4.269629,18.154262,103.806020,85.647206,0.095444,-1.358248
5290,Bennie Jones,"9406 Desiree Rest\r\nSouth Laurafort, ME 72779",43,Male,76-09-04,White,married,Transport-moving,f,9.0,...,0.0,Private,110.054688,45.564053,0.379363,0.133880,1.261706,13.032154,13.294267,202.930061
5291,Bobby Hawk,"41416 Ellis Island\r\nEdgarchester, OR 73214",50,Male,1968-12-02,White,married,Prof-specialty,f,13.0,...,0.0,Local-gov,159.078125,39.412405,-0.363835,1.141354,0.795987,9.677674,18.652776,427.256817
5292,Wally Suggs,"624 Jeremiah Inlet Suite 082\r\nEast Jerome, H...",49,Female,1970-04-05,White,single,Sales,f,9.0,...,0.0,private,116.468750,35.632222,0.386352,2.199301,1.541806,14.173109,12.022914,164.296414


In [1820]:
df_merged['education'].unique()

array([' HS-grad', ' Some-college', ' 7th-8th', ' Bachelors', ' Masters',
       ' 5th-6th', ' Assoc-voc', ' 10th', ' Assoc-acdm', ' Prof-school',
       ' 9th', ' 12th', ' 11th', ' Doctorate', ' 1st-4th', ' Preschool',
       nan], dtype=object)

In [1821]:
cleanse_some_values(df_merged, 'education', [" 1st-4th", " 5th-6th", " 7th-8th", " 9th", " 10th", " 11th", " 12th"], "Primary school")
cleanse_some_values(df_merged, 'education', [" Bachelors", " Masters", " Doctorate"], "College")
cleanse_some_values(df_merged, 'education', [" Assoc-voc", " HS-grad", " Prof-school", " Assoc-acdm", " Some-college"], "Highschool")
df_merged.loc[df_merged['education'] == " Preschool", 'education'] = "Preschool"

In [1822]:
df_merged['workclass'].unique()

array([' Private', ' Self-emp-not-inc', ' Self-emp-inc', 'private', ' ?',
       ' Local-gov', 'self-emp-not-inc', '?', ' Federal-gov', 'local-gov',
       ' State-gov', 'state-gov', 'federal-gov', 'self-emp-inc',
       ' Never-worked', nan, ' Without-pay'], dtype=object)

In [1823]:
cleanse_some_values(df_merged, 'workclass', [" Private"], "private")
cleanse_some_values(df_merged, 'workclass', [" Self-emp-not-inc", " Self-emp-inc", "self-emp-inc", "self-emp-not-inc"], "self-emp")
cleanse_some_values(df_merged, 'workclass', [" Never-worked", " Without-pay"], "not-working")
cleanse_some_values(df_merged, 'workclass', [" Local-gov", "local-gov", " State-gov", "state-gov", " Federal-gov", "federal-gov"], "government")
cleanse_some_values(df_merged, 'workclass', [" ?", "?"], np.nan)

In [1824]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5294 entries, 0 to 5293
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              5294 non-null   object 
 1   address           5294 non-null   object 
 2   age               5294 non-null   int64  
 3   sex               5294 non-null   object 
 4   date_of_birth     5294 non-null   object 
 5   race              4912 non-null   object 
 6   marital-status    5280 non-null   object 
 7   occupation        5276 non-null   object 
 8   pregnant          5275 non-null   object 
 9   education-num     5275 non-null   float64
 10  relationship      5271 non-null   object 
 11  capital-gain      5274 non-null   float64
 12  education         5278 non-null   object 
 13  fnlwgt            5282 non-null   float64
 14  class             5278 non-null   float64
 15  income            5273 non-null   object 
 16  native-country    5281 non-null   object 


In [1825]:
# def change_date_format(data_frame, column):
#     for i in range(len(data_frame[column])):
#         date_format = data_frame[column][i].replace('/', '-')
#         data_frame.loc[i, column] = date_format[:10]
#     return data_frame

def change_date_format(data_frame, column):
    for i in range(len(data_frame[column])):
        date_format = str(data_frame[column][i])
        date_format = date_format.replace('/', '-')
        if date_format[2] == '-' and len(date_format) < 10:
            temp_year = int(date_format[:2])
            if(temp_year <= 20):
                if(temp_year < 10):
                    year = '200'+str(temp_year)
                else:
                    year = '20'+str(temp_year)
            else:
                year = '19'+str(temp_year)
            print(year+date_format[2:10])
            data_frame.loc[i, column] = year+date_format[2:10]
        elif date_format[2] == '-' and len(date_format) == 10:
            day = date_format[:2]
            month = date_format[3:5]
            year = date_format[6:10]
            data_frame.loc[i, column] = year+'-'+month+'-'+day
        else:
            data_frame.loc[i, column] = date_format[:10]
    return data_frame

In [1826]:
df_merged = change_date_format(df_merged,'date_of_birth')

1960-02-18
1952-09-29
1960-09-16
1953-12-20
1970-11-03
1960-06-22
1946-02-27
1983-03-07
1968-08-08
1977-03-22
1967-04-29
1978-04-22
1979-03-18
1967-09-15
1971-03-08
1975-12-11
1960-03-03
1971-02-04
1977-06-08
1959-12-02
1967-03-17
1979-09-07
1980-06-24
1952-10-29
1954-12-20
1969-07-23
1973-11-23
1984-08-04
1955-08-14
1988-01-27
1965-01-29
1981-11-20
1978-01-12
1969-01-12
1970-08-26
1968-07-20
1960-01-20
1952-10-27
1971-10-24
1956-05-02
1962-03-10
1962-11-04
1963-03-12
1965-08-08
1966-12-30
1983-02-14
1974-01-24
1980-08-12
1968-01-02
1979-10-15
1955-02-07
1973-02-08
1942-09-12
1970-02-16
1969-02-06
1972-08-17
1965-11-16
1966-08-20
1959-02-20
1982-02-10
1977-01-22
1971-04-15
1990-08-28
1962-06-15
1953-04-23
1933-09-21
1983-09-16
1971-09-23
1976-05-27
1977-10-31
1965-07-19
1955-04-06
1969-07-15
1938-06-03
1969-06-28
1982-01-29
1950-10-17
1969-03-24
1975-04-01
1957-04-23
1968-10-24
1968-09-21
1957-09-12
1984-10-28
1970-07-29
1975-11-10
1953-07-14
1972-03-03
1967-05-06
1961-04-14
1961-07-05

In [1827]:
date_formats = df_merged.date_of_birth
date_formats = date_formats.str.replace('\d', '0')
date_formats.unique()

array(['0000-00-00'], dtype=object)

In [1828]:
df_merged['date_of_birth']

0       1982-02-08
1       1961-05-04
2       1971-02-03
3       1973-12-23
4       1963-08-22
           ...    
5289    1959-09-21
5290    1976-09-04
5291    1968-12-02
5292    1970-04-05
5293    1944-03-24
Name: date_of_birth, Length: 5294, dtype: object

In [1829]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5294 entries, 0 to 5293
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              5294 non-null   object 
 1   address           5294 non-null   object 
 2   age               5294 non-null   int64  
 3   sex               5294 non-null   object 
 4   date_of_birth     5294 non-null   object 
 5   race              4912 non-null   object 
 6   marital-status    5280 non-null   object 
 7   occupation        5276 non-null   object 
 8   pregnant          5275 non-null   object 
 9   education-num     5275 non-null   float64
 10  relationship      5271 non-null   object 
 11  capital-gain      5274 non-null   float64
 12  education         5278 non-null   object 
 13  fnlwgt            5282 non-null   float64
 14  class             5278 non-null   float64
 15  income            5273 non-null   object 
 16  native-country    5281 non-null   object 


In [1830]:
# year and age must be string and not NaN
def age_from_year(data_frame, year_column, age_column):
    for i in range(len(data_frame[year_column])):
        print(data_frame[year_column][i])
        year = re.sub(r'[^0-9]', '', data_frame[year_column][i])
        year = year[:4]            
        new_age = 2020 - int(year)
        data_frame.loc[i, age_column] = str(new_age)
        print(str(new_age))
    return data_frame

In [1831]:
df_merged = age_from_year(df_merged, 'date_of_birth', 'age')

1982-02-08
38
1961-05-04
59
1971-02-03
49
1973-12-23
47
1963-08-22
57
1964-03-04
56
1960-02-18
60
1979-10-28
41
1977-10-18
43
1962-10-15
58
1970-07-31
50
1952-09-29
68
1960-09-16
60
1953-12-20
67
1943-05-05
77
1965-07-16
55
1966-10-07
54
1961-07-20
59
1972-04-10
48
1971-02-22
49
1984-06-26
36
1982-01-31
38
1972-02-19
48
1967-08-16
53
1962-10-17
58
1955-09-05
65
1951-08-29
69
1976-07-02
44
1976-01-31
44
1954-07-20
66
1987-11-20
33
1983-12-01
37
1958-11-10
62
1976-09-09
44
1953-11-22
67
1954-11-23
66
1963-05-06
57
1955-11-04
65
1976-08-24
44
1953-05-10
67
1962-12-04
58
1974-12-25
46
1981-08-30
39
1965-11-25
55
1968-01-19
52
1965-11-15
55
1958-03-03
62
1979-12-26
41
1976-04-07
44
1953-01-25
67
1970-11-03
50
1969-02-20
51
1960-06-22
60
1974-09-09
46
1946-02-27
74
1945-12-07
75
1971-09-29
49
1966-12-31
54
1983-03-07
37
1964-04-27
56
1962-04-11
58
1980-10-12
40
1961-05-27
59
1977-07-16
43
1955-03-16
65
1962-01-01
58
1961-04-14
59
1968-04-24
52
1966-12-04
54
1962-01-24
58
1962-07-04
58
1970-0

50
1963-12-05
57
1939-06-21
81
1969-12-09
51
1968-07-13
52
1963-10-29
57
1962-11-04
58
1966-03-23
54
1974-09-01
46
1980-11-14
40
1961-10-04
59
1978-09-02
42
1960-12-30
60
1962-06-08
58
1958-07-30
62
2000-05-03
20
1953-11-05
67
1981-12-26
39
1967-03-12
53
1949-12-07
71
1976-05-25
44
1969-02-06
51
1951-10-14
69
1978-08-14
42
1948-04-29
72
1977-01-14
43
1972-08-17
48
1973-11-01
47
1971-03-30
49
1967-11-20
53
1969-02-23
51
1965-06-10
55
1961-11-07
59
1976-07-27
44
1970-09-01
50
1946-05-27
74
1965-11-16
55
1965-12-21
55
1971-02-12
49
1963-08-04
57
1979-08-08
41
1968-02-27
52
1969-12-04
51
1969-08-05
51
1954-03-02
66
1961-11-11
59
1977-02-03
43
1962-12-12
58
1963-10-01
57
1971-03-20
49
1952-07-22
68
1977-07-06
43
1974-06-06
46
1982-05-15
38
1965-09-22
55
1955-11-07
65
1966-09-13
54
1981-06-12
39
1949-03-23
71
1951-03-01
69
1981-02-05
39
1973-04-19
47
1976-12-07
44
1945-12-08
75
1959-12-04
61
1976-07-19
44
1967-05-30
53
1997-12-05
23
1966-08-20
54
1958-02-04
62
1963-10-09
57
1955-02-10
65
198

1970-12-16
50
1963-10-23
57
1959-07-11
61
1955-02-09
65
1971-11-29
49
1965-11-08
55
1963-07-29
57
1961-07-21
59
1975-08-20
45
1952-03-28
68
1967-05-16
53
1963-08-12
57
1942-11-02
78
1969-08-07
51
1967-05-03
53
1969-01-03
51
1979-10-08
41
1968-04-20
52
1967-03-27
53
1972-05-14
48
1951-05-24
69
1973-03-08
47
1974-10-29
46
1957-02-10
63
1961-05-31
59
1961-06-08
59
1967-11-19
53
1979-11-08
41
1965-02-08
55
1973-07-02
47
1962-09-17
58
1979-04-20
41
1972-08-18
48
1973-10-14
47
1958-01-19
62
1956-12-02
64
1965-04-13
55
1989-02-06
31
1953-02-15
67
1968-11-11
52
1942-01-23
78
1966-05-23
54
1974-07-11
46
1955-09-20
65
1979-02-16
41
1954-09-08
66
1973-04-02
47
1977-09-14
43
1948-02-01
72
1982-03-08
38
1970-12-02
50
1966-09-18
54
1965-03-01
55
1965-12-20
55
1959-03-21
61
1992-09-29
28
1962-05-07
58
1967-02-26
53
1968-07-07
52
1969-06-25
51
1959-12-07
61
1964-11-22
56
1972-08-22
48
1967-07-29
53
1970-02-24
50
1965-06-28
55
1947-11-11
73
1970-09-01
50
1951-06-14
69
1971-06-02
49
1950-08-25
70
1950-0

1945-02-13
75
1980-07-02
40
1955-05-30
65
1973-08-23
47
1993-05-05
27
1952-03-22
68
1958-02-20
62
1961-10-13
59
1978-11-14
42
1960-05-18
60
1963-09-17
57
1962-10-26
58
1962-03-27
58
1964-02-25
56
1973-10-14
47
1969-10-26
51
1978-04-21
42
1970-12-31
50
1968-10-12
52
1959-01-20
61
1968-08-24
52
1959-01-25
61
1950-01-31
70
1974-01-07
46
1974-08-12
46
1958-10-27
62
1958-09-23
62
1986-02-25
34
1951-04-04
69
1958-01-18
62
1979-09-11
41
1972-10-04
48
1974-07-22
46
1966-04-16
54
1976-03-26
44
1963-05-20
57
1971-10-05
49
1981-07-14
39
1981-10-12
39
1943-11-29
77
1956-05-02
64
1965-04-05
55
1952-01-25
68
1965-08-12
55
1986-02-15
34
1976-08-11
44
1972-10-27
48
1957-11-29
63
1980-07-02
40
1962-04-07
58
1953-05-27
67
2020-04-26
0
1973-07-14
47
1975-02-09
45
1969-03-22
51
1969-02-26
51
1964-10-07
56
1960-03-26
60
1989-09-29
31
1951-09-22
69
1958-11-19
62
1990-02-22
30
1977-08-08
43
1948-02-23
72
1952-07-05
68
1967-12-03
53
1965-01-18
55
1973-01-12
47
1966-06-01
54
1946-11-25
74
1970-03-10
50
1963-01

50
1990-08-13
30
1974-08-06
46
1952-12-19
68
1986-06-07
34
1963-08-05
57
1967-03-02
53
1979-10-22
41
1961-08-05
59
1978-12-21
42
1946-09-07
74
1977-08-03
43
1969-08-06
51
1963-03-01
57
1954-01-23
66
1984-06-03
36
1952-09-15
68
1961-12-11
59
1953-08-24
67
1969-01-18
51
1969-07-08
51
1962-06-01
58
1960-05-01
60
1961-05-27
59
1975-12-04
45
1992-02-09
28
1976-12-08
44
1984-09-22
36
1952-02-09
68
1966-07-08
54
1980-09-14
40
1982-08-31
38
1973-02-28
47
1966-03-02
54
1966-08-17
54
1978-09-01
42
1985-03-26
35
1967-10-04
53
1952-05-13
68
1956-06-24
64
1955-11-24
65
1976-01-10
44
1954-08-13
66
1970-11-11
50
1979-01-13
41
1978-07-03
42
1979-08-29
41
2001-09-12
19
1968-11-10
52
1969-12-09
51
1947-06-02
73
1979-05-16
41
1997-01-25
23
1969-07-06
51
1975-03-08
45
1960-12-04
60
1942-12-23
78
1976-12-01
44
1941-01-11
79
1979-07-31
41
1961-05-02
59
1972-05-01
48
1979-06-24
41
1979-07-18
41
1954-03-11
66
1976-12-18
44
1972-06-07
48
1963-06-11
57
1983-04-14
37
1936-02-06
84
1956-03-14
64
1983-06-01
37
195

56
1951-07-14
69
1974-03-22
46
1992-04-26
28
1981-11-29
39
1968-08-10
52
1955-09-28
65
1971-03-26
49
1980-10-30
40
1961-08-16
59
1961-09-04
59
1966-02-19
54
1966-01-07
54
1960-09-25
60
1965-07-13
55
1958-02-08
62
1953-08-29
67
1980-10-13
40
1970-01-26
50
1967-02-16
53
1949-05-07
71
1964-06-17
56
1967-03-22
53
1964-04-16
56
1955-11-28
65
1966-11-24
54
1955-09-21
65
1983-08-08
37
1974-10-22
46
1976-06-21
44
1967-06-26
53
1949-01-19
71
1981-06-21
39
1955-03-05
65
1966-09-26
54
1978-03-19
42
1961-10-27
59
1965-12-27
55
1975-11-06
45
1983-12-30
37
1978-10-31
42
1982-05-28
38
1975-07-24
45
1978-11-27
42
1982-03-03
38
1974-02-12
46
1960-12-30
60
1956-03-06
64
1965-07-24
55
1967-06-08
53
1998-08-24
22
1980-08-26
40
1952-09-10
68
1963-10-19
57
1966-12-27
54
1967-02-22
53
1972-05-19
48
1966-05-22
54
1974-09-12
46
1973-01-10
47
2005-08-19
15
1938-03-02
82
1966-03-20
54
1966-08-09
54
1971-08-16
49
1976-04-14
44
1941-06-09
79
1959-01-02
61
1971-06-11
49
1974-10-12
46
1986-09-08
34
1972-11-07
48
197

61
1977-03-09
43
1933-05-14
87
1965-03-04
55
1976-07-11
44
1996-06-13
24
1961-11-07
59
1961-11-07
59
1960-08-08
60
1954-11-13
66
1967-06-25
53
1957-05-29
63
1964-07-22
56
1964-07-22
56
1978-04-26
42
1958-09-13
62
1964-06-16
56
1965-02-28
55
1971-12-03
49
1971-12-03
49
1971-12-03
49
1967-09-12
53
1957-06-03
63
1970-08-29
50
1984-11-09
36
1939-04-27
81
1939-04-27
81
1998-02-04
22
1998-02-04
22
1966-11-29
54
1964-05-13
56
1996-08-19
24
1970-09-23
50
1984-12-13
36
1976-02-09
44
1981-07-24
39
1984-10-26
36
1983-08-05
37
1975-07-28
45
1963-02-09
57
1963-02-09
57
1980-12-28
40
1963-06-13
57
1963-06-13
57
1957-09-27
63
1961-07-17
59
1965-10-21
55
1974-09-07
46
1961-09-02
59
1968-05-25
52
1986-04-08
34
1969-05-28
51
1960-08-22
60
1968-03-18
52
1961-07-09
59
1961-05-21
59
1981-03-10
39
1957-11-24
63
1956-01-01
64
1959-07-27
61
1957-01-27
63
1966-02-08
54
1955-09-13
65
1977-04-01
43
1977-04-01
43
1967-03-31
53
1933-11-26
87
1970-11-29
50
1968-07-20
52
1980-09-07
40
1974-04-05
46
1962-07-26
58
197

42
1950-03-25
70
1965-10-07
55
1971-03-22
49
1977-02-21
43
1969-06-02
51
1976-03-07
44
1975-08-27
45
1936-04-29
84
1973-10-18
47
1961-04-17
59
1952-02-23
68
1970-09-11
50
1968-12-30
52
1974-10-12
46
1988-03-31
32
1984-08-13
36
1970-10-09
50
1980-12-31
40
1979-11-28
41
1979-09-23
41
1952-10-10
68
1983-08-06
37
1962-02-01
58
1962-02-01
58
1978-11-08
42
1977-11-06
43
1963-01-15
57
1972-08-26
48
1972-12-24
48
2004-08-17
16
1961-10-20
59
1956-11-29
64
1985-01-22
35
1971-04-03
49
1968-10-04
52
1967-08-31
53
1961-03-31
59
1973-06-19
47
1973-06-19
47
1965-10-06
55
1965-07-26
55
1967-02-08
53
1990-12-28
30
1954-10-18
66
1972-09-17
48
1982-04-08
38
1980-12-28
40
1958-03-01
62
1951-07-08
69
1962-12-24
58
1974-01-31
46
1981-03-22
39
1955-01-06
65
1967-07-14
53
1953-01-31
67
1955-04-28
65
1961-12-17
59
1962-07-11
58
1976-06-04
44
1948-04-01
72
1958-03-10
62
1967-06-08
53
2001-06-20
19
1976-08-16
44
1984-01-27
36
1964-07-19
56
1973-10-12
47
1968-12-18
52
1971-02-06
49
1979-09-12
41
1964-02-05
56
195

In [1832]:
df_merged

,name,address,age,sex,date_of_birth,race,marital-status,occupation,pregnant,education-num,...,capital-loss,workclass,mean_glucose,std_glucose,kurtosis_glucose,skewness_glucose,mean_oxygen,std_oxygen,kurtosis_oxygen,skewness_oxygen
0,Aaron Davis,"379 Rodriguez Inlet\r\nFisherbury, MD 83114",38,Female,1982-02-08,White,single,Other-service,f,9.0,...,0.0,private,130.640625,NaN,-0.278638,-0.200794,11.642977,41.904978,3.602677,11.954904
1,Aaron Dini,"105 Medina Springs Suite 462\r\nEmilyland, KS ...",59,Female,1961-05-04,White,single,Other-service,f,9.0,...,0.0,private,99.828125,47.239766,0.469564,0.767540,2.816054,16.659173,7.999821,78.263172
2,Aaron Gann,59685 Thomas Extension Apt. 142\r\nFieldsborou...,49,Male,1971-02-03,White,married,Handlers-cleaners,f,10.0,...,0.0,private,92.773438,34.005793,0.602241,3.485345,1.086957,12.610281,14.133424,224.144386
3,Aaron Gregg,"10883 Williams Locks Suite 821\r\nEast Carmen,...",47,Male,1973-12-23,White,married,Farming-fishing,f,4.0,...,0.0,self-emp,101.695312,47.525414,0.402180,0.239359,2.717391,19.675471,8.112776,71.312964
4,Aaron Lindley,"564 Stevenson Park Apt. 839\r\nRodriguezville,...",57,Male,1963-08-22,White,single,Transport-moving,f,10.0,...,0.0,private,114.835938,51.926146,-0.009627,-0.528806,1.227425,12.031249,14.087309,234.473686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,Arturo Mckee,"64582 Mcgee Orchard\r\nBridgeschester, VT 81156",61,Male,1959-09-21,White,married,Prof-specialty,f,15.0,...,0.0,private,28.195312,3921.376883,4.269629,18.154262,103.806020,85.647206,0.095444,-1.358248
5290,Bennie Jones,"9406 Desiree Rest\r\nSouth Laurafort, ME 72779",44,Male,1976-09-04,White,married,Transport-moving,f,9.0,...,0.0,private,110.054688,45.564053,0.379363,0.133880,1.261706,13.032154,13.294267,202.930061
5291,Bobby Hawk,"41416 Ellis Island\r\nEdgarchester, OR 73214",52,Male,1968-12-02,White,married,Prof-specialty,f,13.0,...,0.0,government,159.078125,39.412405,-0.363835,1.141354,0.795987,9.677674,18.652776,427.256817
5292,Wally Suggs,"624 Jeremiah Inlet Suite 082\r\nEast Jerome, H...",50,Female,1970-04-05,White,single,Sales,f,9.0,...,0.0,private,116.468750,35.632222,0.386352,2.199301,1.541806,14.173109,12.022914,164.296414


In [1833]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5294 entries, 0 to 5293
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              5294 non-null   object 
 1   address           5294 non-null   object 
 2   age               5294 non-null   object 
 3   sex               5294 non-null   object 
 4   date_of_birth     5294 non-null   object 
 5   race              4912 non-null   object 
 6   marital-status    5280 non-null   object 
 7   occupation        5276 non-null   object 
 8   pregnant          5275 non-null   object 
 9   education-num     5275 non-null   float64
 10  relationship      5271 non-null   object 
 11  capital-gain      5274 non-null   float64
 12  education         5278 non-null   object 
 13  fnlwgt            5282 non-null   float64
 14  class             5278 non-null   float64
 15  income            5273 non-null   object 
 16  native-country    5281 non-null   object 
